# Building a Spam Filter with Naive Bayes

The aim of this project is to teach the computer to classify messages, to do this we will use the multinominal Naive Bayes algorithm along with a dataset with 5572 SMS messages that have already been classified by humans. 

We will use the dataset collected by Tiago A. Almeida and José María Gómez Hidalgo available at:
https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

Please note this is a real dataset and as such there may be some innapropriate content of individual messages.

## Exploring the Dataset

In [76]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

# Some preliminary info about the dataset
print(sms_spam.shape)
sms_spam.describe()

(5572, 2)


,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [77]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [78]:
sms_spam['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

From this we can see our dataset contains 5572 SMS messages of which the vast majority 4825 or 87% are 'ham'  whilst 747 or 13% are classified as 'spam'.

We will use this dataset to train a Naive Bayes algorithm to build an effective spam filter.

## Training and Test Set

We are going to use 80% (4458 SMS') of the dataset for training and 20% (1114 SMS') for testing.

Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.

In [79]:
# Randomising the entire dataset
sms_randomised = sms_spam.sample(frac=1, random_state=1)

In [80]:
# cut off index between train and testing datasets
index = round(len(sms_randomised) * 0.8)
print(index)

# splitting into training and testing sets
training_set = sms_randomised[:index].reset_index(drop=True)
test_set = sms_randomised[index:].reset_index(drop=True)

# sanity check, expecting size of 4458 SMS'
training_set.describe()

4458


,Label,SMS
count,4458,4458
unique,2,4172
top,ham,"Sorry, I'll call later"
freq,3858,21


We have split the total dataset into two containing data for training and data for testing. We now need to check that the percentage of ham and spam in both new datasets is broadly sinilar to that of the full dataset.

In [81]:
training_set['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

The training set has the same 87% ham to 13% spam ratio as the original dataset.

In [82]:
test_set['Label'].value_counts()

ham     967
spam    147
Name: Label, dtype: int64

And so does the test set, this is as expected given we randomised the dataset properly.

## Letter Case and Punctuation

Before we can effectively train the Naive Bayes algorithm we need to do some data cleaning in order to bring the data in a format that will allow us to extract easily all the information needed. 

We are going to remove the 'SMS' column and replace it with a column for each unique word that appears in the vocabularly. The value in each column will represent how many times that word occurs in that message.

For example: The message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!' which is already classified as spam will have a row that looks like ['spam', 2, 2, 1, 1, 0....0]
where the zeros are all the words in the vocabiulary that don't appear.

To do this we are first going to remove punctuation and bringing all words to lower case.

In [83]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [84]:
# replaces all punctuation with a space then sets all to lowercase
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()

training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [85]:
test_set['SMS'] = test_set['SMS'].str.replace('\W', ' ')
test_set['SMS'] = test_set['SMS'].str.lower()

test_set.head()

,Label,SMS
0,ham,later i guess i needa do mcat study too
1,ham,but i haf enuff space got like 4 mb
2,spam,had your mobile 10 mths update to latest oran...
3,ham,all sounds good fingers makes it difficult ...
4,ham,all done all handed in don t know if mega sh...


## Creating the vocabulary

We need to create a list containing each unique word across all messages. We will do this by splitting each message using the series.str.split() method, iterating over each SMS and then using the set() function to remove any duplicates from the vocabulary list.

In [86]:
# original empty list to be appended
vocabulary = []

# turns each message into a list and makes lower
training_set['SMS'] = training_set['SMS'].str.split()

# Builds the vocabulary list
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
      
    
# Removes duplicate words then saves as a list 
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

print(len(vocabulary))

7783


## The Final Training Set

Eventually we will need to create a new DataFrame, before that however we will first build a disctionary that we will then convert to the DataFrame we need.

In [87]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [88]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,sae,hurting,calm,inch,become,0796xxxxxx,28days,settings,system,lov,...,swiss,conacted,leg,seperated,area,rupaul,kaila,season,txts,bang
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,sae,hurting,calm,inch,become,0796xxxxxx,28days,settings,...,swiss,conacted,leg,seperated,area,rupaul,kaila,season,txts,bang
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants

There are some constants that we will use repeatedly in the naive bayes algorithm which we will calculate here, they include:

- P(Spam), P(ham) the probability message is spam/ham
- N_spam, N_ham, N_vocabulary the respective number of words in all spam, ham and the total vocabulary
- alpha = 1, the Laplace smoothing constant to avoid 0 errors.

In [90]:
# Calculates the total number of spam and ham messages.

num_spam = training_set_clean[training_set_clean['Label'] == 'spam']
num_ham = training_set_clean[training_set_clean['Label'] == 'ham']

In [91]:
# Calculates the probability of a message being spam or ham
# Using formula: num_successful_outcomes / total_outcomes

p_spam = len(num_spam) / len(training_set_clean)
p_ham = len(num_ham) / len(training_set_clean)

print('P(spam) =', p_spam)
print('P(ham) = ', p_ham)

P(spam) = 0.13458950201884254
P(ham) =  0.8654104979811574


In [92]:
# calculates the total number of words contained in spam and ham messages
n_per_spam =  num_spam['SMS'].apply(len)
n_spam = sum(n_per_spam)

n_per_ham = num_ham['SMS'].apply(len)
n_ham = sum(n_per_ham)

print(n_spam)
print(n_ham)

15190
57237


In [93]:
# Number of unique words in the vocabulary
n_vocabulary = len(vocabulary)
print(n_vocabulary)

# Laplace smoothing constant
alpha = 1

7783


## Calculating Parameters

For the Naive Bayes theorem we need P('word_1'|Spam) etc i.e the probably of a word given the message is spam, the value of this will be constant regardless of which message it is in. We therefore need to do 15566 probability calculations (twice the size of the vocabulary) one for both spam and ham. Once these probabilities are known, we can use them along with the constants calculated above to create the spam filter. 

This is one of the benefits of the Naive Bayes algorithm, the main computational stage is already done such that it is very fast at classifying new incoming messages.

In [94]:
# Initializing empty dictionaries to be appended. These will include all words in the dictionary with the
# corresponding number of times it appears in spam and ham messages.

parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

In [95]:
# Isolating the classified spam and ham messages in the cleaned training set

training_spam = training_set_clean[training_set_clean['Label'] == 'spam']
training_ham = training_set_clean[training_set_clean['Label'] == 'ham']

In [96]:
# Calculating the probabilities of each word appearing in spam and ham respectively.
for word in vocabulary:
    # calculates number of time word appears in previous spam messages
    n_word_spam = num_spam[word].sum()
    
    # then calculates the probability of it appearing
    p_word_in_spam = (n_word_spam + alpha) / (n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_in_spam
    
    # same process for ham
    n_word_ham = num_ham[word].sum()
    p_word_in_ham = (n_word_ham + alpha) / (n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_in_ham

## Classifying a New Message

Now to build the function which will classify each message.

In [97]:
import re

def classify(message):

    # Cleans the data so there is no punctuation and all lower case
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Using initial values as the average values for spam/ham
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Updates the probability of the message being spam/ham by calculating probabilities of individual words
    # appearing in both spam/ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    # Classifies the message by comparing respective spam and ham probabilities
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [98]:
# Testing the function on a new message
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [99]:
classify('Sounds good, Tom, then see u there')

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring accuracy

We have now built the spam filter by using the Naive Bayes method. We now need to use the remaining 20% of the original dataset to test the model and to measure its performance.

To do this we will make alterations to the classify function above to return either 'spam' or 'ham'. We can measure the accuracy by comparing the models outputs to the given classification that is already there on the original dataset.

In [102]:
def classify_test_set(message):

    # Cleans the data so there is no punctuation and all lower case
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Using initial values as the average values for spam/ham
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Updates the probability of the message being spam/ham by calculating probabilities of individual words
    # appearing in both spam/ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    # Classifies the message by comparing respective spam and ham probabilities
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'Equal proabilities, have a human classify this!'

In [103]:
# Applying the Naive Bayes algorithm to the testing set, creating new column to compare model output to known classification
test_set['Model output'] = test_set['SMS'].apply(classify_test_set)

test_set.head()

,Label,SMS,Model output
0,ham,later i guess i needa do mcat study too,ham
1,ham,but i haf enuff space got like 4 mb,ham
2,spam,had your mobile 10 mths update to latest oran...,spam
3,ham,all sounds good fingers makes it difficult ...,ham
4,ham,all done all handed in don t know if mega sh...,ham


In [107]:
# Measuring the accuracy of the Spam Filter

# Compares label to model output and increments correct variable. accuracy is correct / size of test set
correct = 0
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['Model output']:
        correct+=1

accuracy = correct / len(test_set)
print('Correct:', correct)
print('Incorrect:', len(test_set) - correct)
print('Accuracy:', accuracy)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The model has proven highly effective at correctly classifying spam with an accuracy rate of 98.7%.

## Future Improvements

We can improve the model by firstly looking to see why the 14  messages were incorrectly classified, we can also improve the model by making the messages case sensitive, this will greatly increase the size of the vocabulary and thus the computation time, but it is expected to make the model more accurate.